### Define team lookup
Shared layers allow a model to use the same weight matrix for multiple steps. In this exercise, you will build a "team strength" layer that represents each team by a single number. You will use this number for both teams in the model. The model will learn a number for each team that works well both when the team is team_1 and when the team is team_2 in the input data.   

The games_season DataFrame is available in your workspace.  

### Instructions
Count the number of unique teams.    
Create an embedding layer that maps each team ID to a single number representing that team's strength.   
The output shape should be 1 dimension (as we want to represent the teams by a single number).   
The input length should be 1 dimension (as each team is represented by exactly one id).

In [ ]:
# Imports
from keras.layers import Embedding
from numpy import unique

# Count the unique number of teams
n_teams = unique(games_season['team_1']).shape[0]

# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,
                        output_dim=1,
                        input_length=1,
                        name='Team-Strength')

### Define team model
The team strength lookup has three components: an input, an embedding layer, and a flatten layer that creates the output.   

If you wrap these three layers in a model with an input and output, you can re-use that stack of three layers at multiple places.   

Note again that the weights for all three layers will be shared everywhere we use them.  

### Instructions
Create a 1D input layer for the team ID (which will be an integer). Be sure to set the correct input shape!   
Pass this input to the team strength lookup layer you created previously.   
Flatten the output of the team strength lookup.  
Create a model that uses the 1D input as input and flattened team strength as output.

In [ ]:
# Imports
from keras.layers import Input, Embedding, Flatten
from keras.models import Model

# Create an input layer for the team ID
teamid_in = Input(shape=(1,))

# Lookup the input in the team strength embedding layer
# team_lookup is a pre-defined embedding layer
strength_lookup = team_lookup(teamid_in)

# Flatten the output
strength_lookup_flat = Flatten()(strength_lookup)

# Combine the operations into a single, re-usable model
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')

### Defining two inputs
In this exercise, you will define two input layers for the two teams in your model. This allows you to specify later in the model how the data from each team will be used differently.

### Instructions
Create an input layer to use for team 1. Recall that our input dimension is 1.   
Name the input "Team-1-In" so you can later distinguish it from team 2.  
Create an input layer to use for team 2, named "Team-2-In".

In [ ]:
# Load the input layer from keras.layers
from keras.layers import Input

# Input layer for team 1
team_in_1 = Input(shape=(1, ), name='Team-1-In' )

# Separate input layer for team 2
team_in_2 = Input(shape=(1, ), name='Team-2-In')

### Lookup both inputs in the same model
Now that you have a team strength model and an input layer for each team, you can lookup the team inputs in the shared team strength model. The two inputs will share the same weights.   

In this dataset, you have 10,888 unique teams. You want to learn a strength rating for each team, such that if any pair of teams plays each other, you can predict the score, even if those two teams have never played before. Furthermore, you want the strength rating to be the same, regardless of whether the team is the home team or the away team.  

To achieve this, you use a shared layer, defined by the re-usable model (team_strength_model()) you built in exercise 3 and the two input layers (team_in_1 and team_in_2) from the previous exercise, all of which are available in your workspace.

### Instructions
Lookup the first team ID in the team strength model.  
Lookup the second team ID in the team strength model.

In [ ]:
# Lookup team 1 in the team strength model
team_1_strength = team_strength_model(team_in_1)

# Lookup team 2 in the team strength model
team_2_strength = team_strength_model(team_in_2)

### Output layer using shared layer
Now that you've looked up how "strong" each team is, subtract the team strengths to determine which team is expected to win the game.  

This is a bit like the seeds that the tournament committee uses, which are also a measure of team strength. But rather than using seed differences to predict score differences, you'll use the difference of your own team strength model to predict score differences.  

The subtract layer will combine the weights from the two layers by subtracting them.

### Instructions
Import the Subtract layer from keras.layers.  
Combine the two-team strength lookups you did earlier.

In [ ]:
# Import the Subtract layer from keras
from keras.layers import Subtract

# Create a subtract layer using the inputs from the previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

### Model using two inputs and one output
Now that you have your two inputs (team id 1 and team id 2) and output (score difference), you can wrap them up in a model so you can use it later for fitting to data and evaluating on new data.  

Your model will look like the following diagram:  

### Instructions
Define a model with the two teams as inputs and use the score difference as the output.   
Compile the model with the 'adam' optimizer and 'mean_absolute_error' loss.

In [ ]:
# Imports
from keras.layers import Subtract
from keras.models import Model

# Subtraction layer from previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

# Create the model
model = Model([team_in_1, team_in_2], score_diff)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

### Fit the model to the regular season training data
Now that you've defined a complete team strength model, you can fit it to the basketball data! Since your model has two inputs now, you need to pass the input data as a list.   

### Instructions
Assign the 'team_1' and 'team_2' columns from games_season to input_1 and input_2, respectively.   
Use 'score_diff' column from games_season as the target.  
Fit the model using 1 epoch, a batch size of 2048, and a 10% validation split.


In [ ]:
# Get the team_1 column from the regular season data
input_1 = games_season['team_1']

# Get the team_2 column from the regular season data
input_2 = games_season['team_2']

# Fit the model to input 1 and 2, using score diff as a target
model.fit([input_1, input_2], 
          games_season['score_diff'],
          epochs=1,
          batch_size=2048,
          validation_split=.1,
          verbose=True)

### Evaluate the model on the tournament test data
The model you fit to the regular season data (model) in the previous exercise and the tournament dataset (games_tourney) are available in your workspace.   

In this exercise, you will evaluate the model on this new dataset. This evaluation will tell you how well you can predict the tournament games, based on a model trained with the regular season data. This is interesting because many teams play each other in the tournament that did not play in the regular season, so this is a very good check that your model is not overfitting.

### Instructions
Assign the 'team_1' and 'team_2' columns from games_tourney to input_1 and input_2, respectively.  
Evaluate the model.

In [ ]:
# Get team_1 from the tournament data
input_1 = games_tourney['team_1']

# Get team_2 from the tournament data
input_2 = games_tourney['team_2']

# Evaluate the model using these inputs
print(model.evaluate([input_1, input_2], games_tourney['score_diff'], verbose=False))

### Make an input layer for home vs. away
Now you will make an improvement to the model you used in the previous chapter for regular season games. You know there is a well-documented home-team advantage in basketball, so you will add a new input to your model to capture this effect.   

This model will have three inputs: team_id_1, team_id_2, and home. The team IDs will be integers that you look up in your team strength model from the previous chapter, and home will be a binary variable, 1 if team_1 is playing at home, 0 if they are not.   

The team_strength_model you used in the previous chapter has been loaded into your workspace. After applying it to each input, use a Concatenate layer to join the two team strengths and with the home vs away variable, and pass the result to a Dense layer.

### Instructions
Create three inputs layers of shape 1, one each for team 1, team 2, and home vs away.   
Lookup the team inputs in team_strength_model().   
Concatenate the team strengths with the home input and pass to a Dense layer.

In [ ]:
# Create an Input for each team
team_in_1 = Input(shape=(1,), name='Team-1-In')
team_in_2 = Input(shape=(1,), name='Team-2-In')

# Create an input for home vs away
home_in = Input(shape=(1,), name='Home-In')

# Lookup the team inputs in the team strength model
team_1_strength = team_strength_model(team_in_1)
team_2_strength = team_strength_model(team_in_2)

# Combine the team strengths with the home input using a Concatenate layer, then add a Dense layer
out = Concatenate()([team_1_strength, team_2_strength, home_in])
out = Dense(1)(out)

### Make a model and compile it
Now that you've input and output layers for the 3-input model, wrap them up in a Keras model class, and then compile the model, so you can fit it to data and use it to make predictions on new data.

### Instructions
Create a model using team_in_1, team_in_2, and home_in as inputs and out as the output.  
Compile the model using the 'adam' optimizer and 'mean_absolute_error' as the loss function.

In [ ]:
# Import the model class
from keras.models import Model

# Make a Model
model = Model([team_in_1, team_in_2, home_in], out)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

### Fit the model and evaluate
Now that you've defined a new model, fit it to the regular season basketball data.   

Use the model you fit in the previous exercise (which was trained on the regular season data) and evaluate the model on data for tournament games (games_tourney).

### Instructions
Fit the model to the games_season dataset, using 'team_1', 'team_2' and 'home' columns as inputs, and the 'score_diff' column as the target.   
Fit the model using 1 epoch, 10% validation split and a batch size of 2048.  
Evaluate the model on games_tourney, using the same inputs and outputs.

In [ ]:
# Fit the model to the games_season dataset
model.fit([games_season.team_1, games_season.team_2, games_season.home],
          games_season.score_diff,
          epochs=1,
          verbose=True,
          validation_split=.1,
          batch_size=2048)

# Evaluate the model on the games_tourney dataset
print(model.evaluate([games_tourney.team_1, games_tourney.team_2, games_tourney.home], 
                      games_tourney.score_diff, 
                      verbose=False))

# Imports
import matplotlib.pyplot as plt
from keras.utils import plot_model

# Plot the model
plot_model(model, to_file='model.png')

# Display the image
data = plt.imread('model.png')
plt.imshow(data)
plt.show()

### Add the model predictions to the tournament data
In lesson 1 of this chapter, you used the regular season model to make predictions on the tournament dataset, and got pretty good results! Try to improve your predictions for the tournament by modeling it specifically.  

You'll use the prediction from the regular season model as an input to the tournament model. This is a form of "model stacking."  

To start, take the regular season model from the previous lesson, and predict on the tournament data. Add this prediction to the tournament data as a new column.

### Instructions

Use the model to predict on the games_tourney dataset. The model has three inputs: 'team_1', 'team_2', and 'home' columns. Assign the predictions to a new column, 'pred'.

In [ ]:
games_tourney['pred'] = model.predict([games_tourney.team_1, games_tourney.team_2, games_tourney.home])

### Create an input layer with multiple columns
In this exercise, you will look at a different way to create models with multiple inputs. This method only works for purely numeric data, but its a much simpler approach to making multi-variate neural networks.  

Now you have three numeric columns in the tournament dataset: 'seed_diff', 'home', and 'pred'. In this exercise, you will create a neural network that uses a single input layer to process all three of these numeric inputs.   

This model should have a single output to predict the tournament game score difference.

### Instructions
Create a single input layer with 3 columns.   
Connect this input to a Dense layer with 1 unit.    
Create a model with input_tensor as the input and output_tensor as the output.  
Compile the model with 'adam' as the optimizer and 'mean_absolute_error' as the loss function.

In [ ]:
# Create an input layer with 3 columns
input_tensor = Input(shape=(3, ))

# Pass it to a Dense layer with 1 unit
output_tensor = Dense(1)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

### Fit the model
Now that you've enriched the tournament dataset and built a model to make use of the new data, fit that model to the tournament data.  

Note that this model has only one input layer that is capable of handling all 3 inputs, so it's inputs and outputs do not need to be a list.  

Tournament games are split into a training set and a test set. The tournament games before 2010 are in the training set, and the ones after 2010 are in the test set.

### Instructions
Fit the model to the games_tourney_train dataset using 1 epoch.  
The input columns are 'home', 'seed_diff', and 'pred'.  
The target column is 'score_diff'.

In [ ]:
# Fit the model
model.fit(games_tourney_train[['home', 'seed_diff','pred']],
          games_tourney_train['score_diff'],
          epochs=1,
          verbose=True)

### Evaluate the model
Now that you've fit your model to the tournament training data, evaluate it on the tournament test data. Recall that the tournament test data contains games from after 2010.

### Instructions
Evaluate the model on the games_tourney_test data.
Recall that the model's inputs are 'home', 'seed_diff', and 'prediction' columns and the target column is 'score_diff'.

In [ ]:
# Evaluate the model on the games_tourney_test dataset
print(model.evaluate(games_tourney_test[['home', 'seed_diff', 'prediction']],
                     games_tourney_test['score_diff'], verbose=False))